In [2367]:
import pandas as pd
import numpy as np

In [2368]:
####Invoice Number matching

In [2369]:
ocr_log=pd.read_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ocr_log_set2.csv")
ab_inbev_labels=pd.read_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ab_inbev_paid_invoice.csv",engine="python")

In [2370]:
ab_inbev_labels.columns

Index(['DocumentNo', 'Reference', 'Amount.in.LC', 'Amount', 'Doc..Date'], dtype='object')

In [2371]:
ocr_log.columns

Index(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num',
       'left', 'top', 'width', 'height', 'conf', 'text', 'FileName'],
      dtype='object')

In [2372]:
inv_labels=ab_inbev_labels[["DocumentNo","Reference"]]
inv_labels["Check"]="InvoiceNumber"
inv_labels["length_of_inv"]=inv_labels["Reference"].str.len()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [2375]:
inv_labels.head()

,DocumentNo,InvoiceNumber,Check,length_of_inv
0,2.000001e+09,NaN,InvoiceNumber,NaN
1,5.100013e+09,9945223261,InvoiceNumber,10.0
2,1.900005e+09,9945229827,InvoiceNumber,10.0
3,5.100013e+09,9945230091,InvoiceNumber,10.0
4,5.100013e+09,9945234944,InvoiceNumber,10.0


In [2374]:
inv_labels.rename(columns={'Reference': 'InvoiceNumber'}, inplace=True)


/home/sambid/.local/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [2376]:
inv_labels=inv_labels.drop_duplicates()
inv_labels=inv_labels.dropna()

In [2382]:
#ocr_log.head()
ocr_log["InvoiceNumber"]=ocr_log["text"]
ocr_log["InvoiceNumber"]=ocr_log["InvoiceNumber"].replace(np.nan," ")

In [2389]:
ocr_log["DocumentNumber"]=ocr_log["FileName"].str.split("/").str[-1]
ocr_log["DocumentNumber"]=ocr_log["DocumentNumber"].str.split(".").str[0]

In [2394]:
ocr_log["DocumentNumber"]=pd.to_numeric(ocr_log["DocumentNumber"],errors="coerce")

In [2398]:
inv_labels.rename(columns={"DocumentNo":"DocumentNumber"},inplace=True)

In [2407]:
ocr_log["InvoiceNumber"]=ocr_log["InvoiceNumber"].str.replace('[^\w\s]','')

In [2409]:
#ocr_log.head()

In [2410]:

#inv_labels=inv_labels[inv_labels["length_of_inv"]>=5]
ocr_inv_num=pd.merge(ocr_log,inv_labels,on=["DocumentNumber","InvoiceNumber"],how="left",indicator=True)

In [2411]:
ocr_inv_num["_merge"].value_counts()

left_only     237906
both             416
right_only         0
Name: _merge, dtype: int64

In [2412]:
match=ocr_inv_num[ocr_inv_num["_merge"]=="both"]

In [2415]:
len(ocr_inv_num.DocumentNumber.unique())

436

In [2405]:
ocr_inv_num.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceNumber,DocumentNumber,Check,length_of_inv,_merge
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5.100012e+09,NaN,NaN,left_only
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5.100012e+09,NaN,NaN,left_only
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5.100012e+09,NaN,NaN,left_only
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5.100012e+09,NaN,NaN,left_only
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,hy,5.100012e+09,NaN,NaN,left_only


In [2414]:
ocr_inv_num.to_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ocr_inv_num.csv")

In [364]:
d2["UniqueID"] = d2.groupby("FileName").cumcount()
d2['max_id'] = (d2.groupby(['FileName'])['UniqueID'].transform(max))/2
d2["max_id"]=d2["max_id"].round()
d2.head()

,text,FileName,InvoiceNumber,Check,length_of_inv,UniqueID,max_id
0,NaN,/home/sambid/results/5100011564.pdf1.png,,NaN,NaN,0,6.0
1,NaN,/home/sambid/results/5100011564.pdf1.png,,NaN,NaN,1,6.0
2,NaN,/home/sambid/results/5100011564.pdf1.png,,NaN,NaN,2,6.0
3,NaN,/home/sambid/results/5100011564.pdf1.png,,NaN,NaN,3,6.0
4,hy,/home/sambid/results/5100011564.pdf1.png,hy,NaN,NaN,4,6.0


In [365]:
d2["inv_act"]=np.where(((d2["UniqueID"]<=d2["max_id"]) & (d2["Check"]=="InvoiceNumber")),"InvoiceNumber",np.nan)

In [1872]:
#d2[d2["FileName"]=="/home/sambid/results/5100003058.pdf0.png"]

In [373]:
d2.to_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ocr_log_inv_labels.csv",index=False)

In [ ]:
#####Invoice Date matching

In [2416]:
ocr_log=pd.read_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ocr_log_set2.csv")
ab_inbev_labels=pd.read_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ab_inbev_paid_invoice.csv",engine="python")

In [2418]:
ab_inbev_labels.columns

Index(['DocumentNo', 'Reference', 'Amount.in.LC', 'Amount', 'Doc..Date'], dtype='object')

In [2422]:
#d1=ocr_log[["page_num","block_num","par_num","line_num","text","FileName"]]
date_labels=ab_inbev_labels[["Doc..Date","DocumentNo"]]


date_labels.rename(columns={'DocumentNo': 'DocumentNumber', 'Doc..Date': 'InvoiceDate'}, inplace=True)

date_labels["Check"]="InvoiceDate"
date_labels.head()

/home/sambid/.local/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,InvoiceDate,DocumentNumber,Check
0,1/25/2016,2.000001e+09,InvoiceDate
1,12/1/2015,5.100013e+09,InvoiceDate
2,12/10/2015,1.900005e+09,InvoiceDate
3,12/10/2015,5.100013e+09,InvoiceDate
4,12/14/2015,5.100013e+09,InvoiceDate


In [2423]:
date_labels["InvoiceDate"]=date_labels["InvoiceDate"].str.replace("0:00","")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [2424]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png


In [2426]:
ocr_log["InvoiceDate"]=ocr_log["text"]
ocr_log["InvoiceDate"]=ocr_log["InvoiceDate"].replace(np.nan,"")
#d1.isnull().sum()

In [2427]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,hy


In [461]:
#pattern="\d+/\d+(?:/\d+)?|(?:\d+ )?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[.,]?(?:-\d+-\d+| \d+(?:th|rd|st|nd)?,? \d+| \d+)|(\d+.\d+.\d+)|[\d]{1,2}[-][\w]{3}[-][\d]{2,4}"
#pattern="\d+/\d+(?:/\d+)?|(?:\d+ )?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[.,-]?(?:-\d+-\d+| \d+(?:th|rd|st|nd)?,? \d+| \d+)|[\d]{1,2}[-./][\w]{3}[-./][\d]{2,4}"

In [2428]:
pattern1="([\d]{1,2}[-./][\d]{1,2}[-./][\d]{2,4})|([\d]{1,2}[-][\w]{3}[-][\d]{1,2})|([\d]{1,2}[ ][\w.,]{3,12}[ ][\d]{2,4})"

In [2429]:
ocr_log["date_pattern"] = ocr_log['InvoiceDate'].str.contains(pattern1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [2430]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,date_pattern
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,hy,False


In [2432]:
ocr_log["InvoiceDate"]=np.where(ocr_log["date_pattern"]==True,ocr_log['InvoiceDate'],np.nan)


In [2434]:
#ocr_log[ocr_log["date_pattern"]==True].head(10)

In [2435]:
pattern2="(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)"
ocr_log["month_pattern"]=ocr_log["text"].str.match(pattern2,case=False).shift(0)
ocr_log["month_pattern1"]=ocr_log["text"].str.match(pattern2,case=False).shift(-1).fillna(False)
ocr_log["month_pattern2"]=ocr_log["text"].str.match(pattern2,case=False).shift(1).fillna(False)
ocr_log["month_pattern3"]=ocr_log["text"].str.match(pattern2,case=False).shift(2).fillna(False)

In [2436]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,date_pattern,month_pattern,month_pattern1,month_pattern2,month_pattern3
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,False,False,False
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,False,False,False
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,False,False,False
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,False,False,False
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,NaN,False,False,False,False,False


In [969]:
#d1[d1["FileName"]=="/home/sambid/results/5100005055.pdf1.png"]

In [2437]:
ocr_log.loc[ocr_log['month_pattern'] == True, 'month_regex1'] = ocr_log['text']
ocr_log.loc[ocr_log['month_pattern2'] == True, 'month_regex1'] = ocr_log['text']
ocr_log.loc[ocr_log['month_pattern3'] == True, 'month_regex1'] = ocr_log['text']
ocr_log.loc[ocr_log['month_pattern1'] == True, 'month_regex2'] = ocr_log['text']
ocr_log.loc[ocr_log['month_pattern'] == True, 'month_regex2'] = ocr_log['text']
ocr_log.loc[ocr_log['month_pattern2'] == True, 'month_regex2'] = ocr_log['text']
#d1.loc[d1['date_regex1'] == True, 'month_regex2'] = d1['text']
columns = ["month_pattern","month_pattern1","month_pattern2","month_pattern3"]
ocr_log.drop(columns, inplace=True, axis=1)
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,date_pattern,month_regex1,month_regex2
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,NaN
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,NaN
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,NaN
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,NaN
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,NaN,False,NaN,NaN


In [2441]:
month_1.columns

Index(['page_num', 'block_num', 'par_num', 'line_num', 'FileName',
       'month_regex1'],
      dtype='object')

In [2443]:
month_1=ocr_log[["line_num","FileName","month_regex1"]]
month_1=month_1.dropna()
month_2=ocr_log[["line_num","FileName","month_regex2"]]
month_2=month_2.dropna()

In [2444]:
#month_2[month_2["FileName"].str.contains("/home/sambid/results/1900005055.pdf1.png")]
month_1.head()

,line_num,FileName,month_regex1
204,1,/home/sambid/results/5100005055.pdf1.png,Martim
205,1,/home/sambid/results/5100005055.pdf1.png,Della
206,1,/home/sambid/results/5100005055.pdf1.png,Valle
207,1,/home/sambid/results/5100005055.pdf1.png,March
208,1,/home/sambid/results/5100005055.pdf1.png,"18,"


In [2492]:
month_1.rename(columns={"month_regex1":"month_regex"},inplace=True)
month_2.rename(columns={"month_regex2":"month_regex"},inplace=True)
month=month_1.append(month_2)

In [2445]:

month_3=month_1.groupby(["FileName","line_num"])['month_regex1'].agg(' '.join).reset_index()
#month_4=month_2[["FileName","line_num","month_regex2"]]
month_4=month_2.groupby(["FileName","line_num"])['month_regex2'].agg(' '.join).reset_index()
#month_1.head()

In [2452]:
#month_3.head()

In [2455]:
month_3.rename(columns={'month_regex1': 'month_regex'}, inplace=True)
month_4.rename(columns={'month_regex2': 'month_regex'}, inplace=True)

#month_3.to_csv("/datarepo/sambid/month_regex.csv",index=False)

,FileName,line_num,month_regex
0,/home/sambid/results/1900000244.pdf1.png,4,Separt: Aetour Tarif
1,/home/sambid/results/1900000244.pdf4.png,8,Separts AetOur Tarif/
2,/home/sambid/results/1900000477.pdf1.png,1,"mar, 10-Mar 1"
3,/home/sambid/results/1900000478.pdf1.png,1,"mar, 10-Mar 1"
4,/home/sambid/results/1900000522.pdf0.png,1,February 2016:00:00 Belgiam:


In [2470]:
month_5 = month_3.append(month_4)
month_5.head()

,FileName,line_num,month_regex
0,/home/sambid/results/1900000244.pdf1.png,4,Separt: Aetour Tarif
1,/home/sambid/results/1900000244.pdf4.png,8,Separts AetOur Tarif/
2,/home/sambid/results/1900000477.pdf1.png,1,"mar, 10-Mar 1"
3,/home/sambid/results/1900000478.pdf1.png,1,"mar, 10-Mar 1"
4,/home/sambid/results/1900000522.pdf0.png,1,February 2016:00:00 Belgiam:


In [2471]:
month_5=month_5.drop_duplicates()

In [2505]:
#month.head()

In [2472]:
month_main=pd.merge(month,month_5,on=["FileName","line_num"],how="left")

,FileName,line_num,month_regex
0,/home/sambid/results/1900000244.pdf1.png,4,Separt: Aetour Tarif
1,/home/sambid/results/1900000244.pdf4.png,8,Separts AetOur Tarif/
2,/home/sambid/results/1900000477.pdf1.png,1,"mar, 10-Mar 1"
3,/home/sambid/results/1900000478.pdf1.png,1,"mar, 10-Mar 1"
4,/home/sambid/results/1900000522.pdf0.png,1,February 2016:00:00 Belgiam:


In [2498]:
month_main.rename(columns={"month_regex_x":"Text","month_regex_y":"InvoiceDate"},inplace=True)

In [2503]:
month_main.head()

,line_num,FileName,Text,InvoiceDate
0,1,/home/sambid/results/5100005055.pdf1.png,Martim,"Martim Della Valle March 18, 2016 February 29,..."
1,1,/home/sambid/results/5100005055.pdf1.png,Della,"Martim Della Valle March 18, 2016 February 29,..."
2,1,/home/sambid/results/5100005055.pdf1.png,Valle,"Martim Della Valle March 18, 2016 February 29,..."
3,1,/home/sambid/results/5100005055.pdf1.png,March,"Martim Della Valle March 18, 2016 February 29,..."
4,1,/home/sambid/results/5100005055.pdf1.png,"18,","Martim Della Valle March 18, 2016 February 29,..."


In [2495]:
pattern3="[\w]{3,13}[.]?[ ]*[\d]{2}[ ,thstrnd]+[\d]{4}|[\d]{2}[ ]+[\w]{3,13}[.,]?[ ]+[\d]{4}"

In [2507]:
month_main["month_pattern"] = month_main["InvoiceDate"].str.contains(pattern3)

In [2516]:
month_main=month_main[month_main["month_pattern"]==True]

In [2520]:
pattern4="20[0-9][0-9]$"
month_main["month_pattern"] = month_main["InvoiceDate"].str.contains(pattern4)

In [2528]:
month_main=month_main[month_main["month_pattern"]==True]
month_main.shape

(2133, 5)

In [2477]:
del month_5["month_pattern"]


In [2557]:
ocr_log["InvoiceDate"]=ocr_log["InvoiceDate"].fillna("")
ocr_log["InvoiceDate"]=ocr_log["InvoiceDate"].astype(str)
columns = ["month_regex1","month_regex2"]
ocr_log.drop(columns, inplace=True, axis=1)
ocr_log["DocumentNumber"]=ocr_log["FileName"].str.split("/").str[-1]
ocr_log["DocumentNumber"]=ocr_log["DocumentNumber"].str.split(".").str[0]

ocr_log["InvoiceDate"]=ocr_log["InvoiceDate"].str.replace("Date:","")
x=ocr_log['InvoiceDate'].str.extractall(r'(\d+|\w+)')[0].unstack()

x=x[[0,1,2]]
ocr_log = pd.concat([ocr_log.reset_index(drop=True), x], axis=1)
ocr_log["InvoiceDate1"]=ocr_log[0].astype(str)+'-'+ocr_log[1].astype(str)+'-'+ocr_log[2].astype(str)


,FileName,line_num,month_regex
19,/home/sambid/results/1900000528.pdf2.png,1,DECEMBER 2015 DECEMBER 2016
45,/home/sambid/results/1900002809.pdf0.png,1,"August 12, 2016"
67,/home/sambid/results/3000002125.pdf0.png,1,"March 20, 2018"
68,/home/sambid/results/5100000000 (2).pdf0.png,1,"March 20, 2018"
73,/home/sambid/results/5100000001.pdf0.png,4,"February 10, 2018 June 8, 2018"


In [2558]:
ocr_log["InvoiceDate1"]=

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,date_pattern,0,1,2,InvoiceDate1,DocumentNumber
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,NaN,NaN,NaN,nan-nan-nan,5100011564
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,NaN,NaN,NaN,nan-nan-nan,5100011564
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,NaN,NaN,NaN,nan-nan-nan,5100011564
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,NaN,NaN,NaN,nan-nan-nan,5100011564
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,,False,NaN,NaN,NaN,nan-nan-nan,5100011564


In [2559]:
columns = [0,1,2,"date_pattern"]
ocr_log.drop(columns, inplace=True, axis=1)

In [2561]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,date_pattern,InvoiceDate1,DocumentNumber
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,nan-nan-nan,5100011564
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,nan-nan-nan,5100011564
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,nan-nan-nan,5100011564
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,False,nan-nan-nan,5100011564
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,,False,nan-nan-nan,5100011564


In [2569]:
ocr_log["InvoiceDate1"]=pd.to_datetime(ocr_log["InvoiceDate1"],infer_datetime_format=True,errors="coerce")

In [2571]:
columns=["InvoiceDate","date_pattern"]
ocr_log.drop(columns,axis=1,inplace=True)
ocr_log.rename(columns={"InvoiceDate1":"InvoiceDate"},inplace=True)

In [2572]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,DocumentNumber
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaT,5100011564
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaT,5100011564
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaT,5100011564
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,NaT,5100011564
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,NaT,5100011564


In [2573]:
#ocr_log[ocr_log["date_pattern"]==True].head(10)

In [2574]:

#o

In [767]:
d1["InvoiceDate1"]=pd.to_datetime(d1["InvoiceDate1"],infer_datetime_format=True,errors="coerce")

In [2548]:
ocr_log[ocr_log["date_pattern"]==True].head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,date_pattern,0,1,2,InvoiceDate1
46,5,1,4,2,1,1,107,527,125,23,96,10-01-2016,/home/sambid/results/5100011030.pdf0.png,10-01-2016,True,10,01,2016,10-01-2016
72,5,1,4,4,1,5,1806,1715,134,28,96,10-01-2016,/home/sambid/results/5100011030.pdf0.png,10-01-2016,True,10,01,2016,10-01-2016
483,5,1,9,1,2,1,888,508,295,25,89,BEANH2018-09-001,/home/sambid/results/5100008172.pdf0.png,BEANH2018-09-001,True,BEANH2018,09,001,BEANH2018-09-001
518,5,1,14,1,2,1,1550,507,163,26,96,13-09-2018,/home/sambid/results/5100008172.pdf0.png,13-09-2018,True,13,09,2018,13-09-2018
597,5,1,19,1,3,2,796,1526,123,19,96,30-11-2017,/home/sambid/results/5100008172.pdf0.png,30-11-2017,True,30,11,2017,30-11-2017
617,5,1,20,1,4,2,796,1691,123,19,96,30-11-2017,/home/sambid/results/5100008172.pdf0.png,30-11-2017,True,30,11,2017,30-11-2017
638,5,1,21,1,4,2,796,1855,123,19,96,30-11-2017,/home/sambid/results/5100008172.pdf0.png,30-11-2017,True,30,11,2017,30-11-2017
659,5,1,22,1,4,2,796,2020,123,19,96,30-11-2017,/home/sambid/results/5100008172.pdf0.png,30-11-2017,True,30,11,2017,30-11-2017
671,5,1,23,1,3,2,796,2156,123,19,96,30-11-2017,/home/sambid/results/5100008172.pdf0.png,30-11-2017,True,30,11,2017,30-11-2017
693,5,1,24,1,4,2,796,2320,123,19,96,30-11-2017,/home/sambid/results/5100008172.pdf0.png,30-11-2017,True,30,11,2017,30-11-2017


In [2578]:
date_labels.head()

,InvoiceDate,DocumentNumber,Check
0,2016-01-25,2.000001e+09,InvoiceDate
1,2015-12-01,5.100013e+09,InvoiceDate
2,2015-12-10,1.900005e+09,InvoiceDate
3,2015-12-10,5.100013e+09,InvoiceDate
4,2015-12-14,5.100013e+09,InvoiceDate


In [2577]:
date_labels["InvoiceDate"]=pd.to_datetime(date_labels["InvoiceDate"],infer_datetime_format=True,errors="coerce")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [2579]:
#date_labels=date_labels[["Check","InvoiceDate1"]]
date_labels=date_labels.drop_duplicates()
date_labels.head()

,InvoiceDate,DocumentNumber,Check
0,2016-01-25,2.000001e+09,InvoiceDate
1,2015-12-01,5.100013e+09,InvoiceDate
2,2015-12-10,1.900005e+09,InvoiceDate
3,2015-12-10,5.100013e+09,InvoiceDate
4,2015-12-14,5.100013e+09,InvoiceDate


In [2586]:
date_labels=date_labels.dropna()

In [ ]:
ocr_log["DocumentNumber"]=pd.to_numeric(ocr_log["DocumentNumber"],errors="coerce")

In [2674]:
ocr_inv_date=pd.merge(ocr_log,date_labels,on=["DocumentNumber","InvoiceDate"],how="left",indicator=True)

In [2675]:
ocr_inv_date[ocr_inv_date["Check"]=="InvoiceDate"].head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceDate,DocumentNumber,Check,_merge
46,5,1,4,2,1,1,107,527,125,23,96,10-01-2016,/home/sambid/results/5100011030.pdf0.png,2016-10-01,5.100011e+09,InvoiceDate,both
72,5,1,4,4,1,5,1806,1715,134,28,96,10-01-2016,/home/sambid/results/5100011030.pdf0.png,2016-10-01,5.100011e+09,InvoiceDate,both
518,5,1,14,1,2,1,1550,507,163,26,96,13-09-2018,/home/sambid/results/5100008172.pdf0.png,2018-09-13,5.100008e+09,InvoiceDate,both
1365,5,1,1,1,1,5,2957,62,304,32,92,Date:12/15/2016,/home/sambid/results/5100000327.pdf2.png,2016-12-15,5.100000e+09,InvoiceDate,both
1832,5,1,4,1,2,3,2173,658,140,28,81,04/26/17,/home/sambid/results/5100005009.pdf1.png,2017-04-26,5.100005e+09,InvoiceDate,both


In [2676]:
columns=["_merge","InvoiceDate"]
ocr_inv_date.drop(columns,axis=1,inplace=True)

In [2677]:
ocr_inv_date.shape

(238322, 15)

In [2611]:
del month_main["month_pattern"]

In [2613]:
month_main.rename(columns={"Text":"text"},inplace=True)

In [2665]:
month_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2133 entries, 0 to 7141
Data columns (total 4 columns):
line_num       2133 non-null int64
FileName       2133 non-null object
text           2133 non-null object
InvoiceDate    2133 non-null object
dtypes: int64(1), object(3)
memory usage: 83.3+ KB


In [2668]:
month_main["len_inv_date"]=month_main["InvoiceDate"].str.len()
month_main=month_main[month_main["len_inv_date"]<=18]
del month_main["len_inv_date"]
month_main.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,line_num,FileName,text,InvoiceDate
9,3,/home/sambid/results/5100005055.pdf1.png,February,"February 19, 2016"
10,3,/home/sambid/results/5100005055.pdf1.png,"19,","February 19, 2016"
11,3,/home/sambid/results/5100005055.pdf1.png,2016,"February 19, 2016"
27,3,/home/sambid/results/5100003058.pdf0.png,November,"November 29, 2017"
28,3,/home/sambid/results/5100003058.pdf0.png,"29,","November 29, 2017"


In [2671]:
month_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1245 entries, 9 to 7141
Data columns (total 4 columns):
line_num       1245 non-null int64
FileName       1245 non-null object
text           1245 non-null object
InvoiceDate    1245 non-null object
dtypes: int64(1), object(3)
memory usage: 48.6+ KB


In [2691]:
#month_main["InvoiceDate1"]=pd.datetime(month_main["InvoiceDate"],dayfirst=False,yearfirst=False,errors="coerce")
month_main["InvoiceDate"]=pd.to_datetime(month_main["InvoiceDate"],dayfirst=False,yearfirst=False,errors="coerce")

In [2692]:
month_main=month_main.drop_duplicates()

In [2694]:
del month_main["InvoiceDate1"]

In [2695]:
ocr_inv_date1=pd.merge(ocr_inv_date,month_main,on=["line_num","text","FileName"],how="left",indicator=True)

In [2697]:
ocr_inv_date1.shape

(238322, 17)

In [2698]:
ocr_inv_date1.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,DocumentNumber,Check,InvoiceDate,_merge
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,NaN,NaT,left_only
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,NaN,NaT,left_only
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,NaN,NaT,left_only
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,NaN,NaT,left_only
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,NaN,NaT,left_only


In [2700]:
del ocr_inv_date1["_merge"]
#ocr_inv_date.head()

In [2701]:
ocr_inv_date1=pd.merge(ocr_inv_date1,date_labels,on=["DocumentNumber","InvoiceDate"],how="left",indicator=True)

In [2710]:
ocr_inv_date1.columns

Index(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num',
       'left', 'top', 'width', 'height', 'conf', 'text', 'FileName',
       'DocumentNumber', 'Check_x', 'InvoiceDate', 'Check_y', '_merge'],
      dtype='object')

In [2713]:
#ocr_inv_date1=ocr_inv_date1.fillna("")
ocr_inv_date1["Check_x"] = ocr_inv_date1["Check_x"].replace(np.nan, '', regex=True)
ocr_inv_date1["Check_y"] = ocr_inv_date1["Check_y"].replace(np.nan, '', regex=True)
ocr_inv_date1.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,DocumentNumber,Check_x,InvoiceDate,Check_y,_merge
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaT,,left_only
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaT,,left_only
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaT,,left_only
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaT,,left_only
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaT,,left_only


In [2714]:

ocr_inv_date1["Label"]=ocr_inv_date1["Check_x"].astype(str)+ocr_inv_date1["Check_y"].astype(str)

In [2719]:
columns=["Check_x","Check_y","_merge","InvoiceDate"]
ocr_inv_date1.drop(columns,axis=1,inplace=True)
ocr_inv_date1.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,DocumentNumber,Label
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,


In [2720]:
ocr_inv_date1.to_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ocr_inv_date.csv",index=False)
#d1.head(100)

In [ ]:
################################################################################
##################################################################################

In [2305]:
ocr_log=pd.read_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ocr_log_set2.csv")
ab_labels=pd.read_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ab_inbev_paid_invoice.csv",engine="python")

In [2306]:
ocr_log.columns

Index(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num',
       'left', 'top', 'width', 'height', 'conf', 'text', 'FileName'],
      dtype='object')

In [2350]:
x=ab_labels[["DocumentNo","Amount.in.LC"]]
y=ab_labels[["DocumentNo","Amount"]]
x.columns = ["DocumentNumber","InvoiceAmount"]
y.columns = ["DocumentNumber","InvoiceAmount"]
z=x.append(y)
#z=z.drop_duplicates()

In [2351]:
z.head()

,DocumentNumber,InvoiceAmount
0,2.000001e+09,"1,906,710.01"
1,5.100013e+09,304.27
2,1.900005e+09,"2,424.20"
3,5.100013e+09,152.14
4,5.100013e+09,334.08


In [2352]:
z["InvoiceAmount"]=z["InvoiceAmount"].astype(str)
z["num_1"]=z["InvoiceAmount"].str.split(r'[,.]').str[0]
z["num_2"]=z["InvoiceAmount"].str.split(r'[,.]').str[1]
z["num_3"]=z["InvoiceAmount"].str.split(r'[,.]').str[2]
z.fillna("",inplace=True)

z["InvoiceAmount1"]=z["num_1"].astype(str)+"."+z["num_2"].astype(str)+","+z["num_3"].astype(str)
z["InvoiceAmount2"]=z["num_1"].astype(str)+z["num_2"].astype(str)+"."+z["num_3"].astype(str)
z["InvoiceAmount3"]=z["num_1"].astype(str)+","+z["num_2"].astype(str)+"."+z["num_3"].astype(str)
z["InvoiceAmount4"]=z["InvoiceAmount"].str.replace(".00","")

z["InvoiceAmount1"]=z["InvoiceAmount1"].str.replace("[,]$","")
z["InvoiceAmount2"]=z["InvoiceAmount2"].str.replace("[\.]$","")
z["InvoiceAmount3"]=z["InvoiceAmount3"].str.replace("[\.]$","")

z1=z[["DocumentNumber","InvoiceAmount","InvoiceAmount1","InvoiceAmount2","InvoiceAmount3","InvoiceAmount4"]]
z1=pd.melt(z1, id_vars='DocumentNumber', value_vars=['InvoiceAmount1', 'InvoiceAmount2', 'InvoiceAmount3'])







In [2353]:
z1.rename(columns={"variable":"Check","value":"InvoiceAmount"},inplace=True)

In [2354]:
z1["Check"]="InvoiceNumber"

In [2322]:
#ocr_log.head()

In [2325]:
ocr_log["InvoiceAmount"]=ocr_log["text"]
ocr_log["InvoiceAmount"]=ocr_log["InvoiceAmount"].replace(np.nan,"")

In [2326]:
ocr_log["DocumentNumber"]=ocr_log["FileName"].str.split("/").str[-1]
ocr_log["DocumentNumber"]=ocr_log["DocumentNumber"].str.split(".").str[0]
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceAmount,DocumentNumber
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5100011564
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5100011564
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5100011564
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,,5100011564
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,hy,5100011564


In [2327]:
pattern="^[\$€£]?[ ]*[\d.,]+[ ]*[EUR€£ ]*$"
ocr_log["InvoiceAmount"]=ocr_log["InvoiceAmount"].str.contains(pattern)

In [2329]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceAmount,DocumentNumber
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,False,5100011564


In [2331]:
ocr_log["InvoiceAmount1"]=np.where(ocr_log["InvoiceAmount"]==True,ocr_log["text"],"")
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,InvoiceAmount,DocumentNumber,InvoiceAmount1
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564,
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564,
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564,
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,False,5100011564,
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,False,5100011564,


In [2335]:
ocr_log.columns

Index(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num',
       'left', 'top', 'width', 'height', 'conf', 'text', 'FileName',
       'InvoiceAmount', 'DocumentNumber', 'InvoiceAmount1'],
      dtype='object')

In [2336]:
del ocr_log["InvoiceAmount"]

In [2337]:
ocr_log.rename(columns={"InvoiceAmount1":"InvoiceAmount"},inplace=True)

In [2338]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,DocumentNumber,InvoiceAmount
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,5100011564,


In [2340]:
ocr_log["InvoiceAmount"]=ocr_log["InvoiceAmount"].astype(str)
ocr_log["InvoiceAmount"]=ocr_log["InvoiceAmount"].str.replace("$","")
ocr_log["InvoiceAmount"]=ocr_log["InvoiceAmount"].str.replace("EUR","")
ocr_log["InvoiceAmount"]=ocr_log["InvoiceAmount"].str.replace("€","")
ocr_log["InvoiceAmount"]=ocr_log["InvoiceAmount"].str.replace("£","")


In [2341]:
ocr_log.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,DocumentNumber,InvoiceAmount
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5100011564,
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,5100011564,


In [2342]:
ocr_log["DocumentNumber"]=pd.to_numeric(ocr_log["DocumentNumber"],errors="coerce")

In [2348]:
ocr_log.columns

Index(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num',
       'left', 'top', 'width', 'height', 'conf', 'text', 'FileName',
       'DocumentNumber', 'InvoiceAmount'],
      dtype='object')

In [2349]:
z1.

Index(['DocumentNumber', 'Check', 'InvoiceNumber'], dtype='object')

In [2355]:
ocr_bsak_match=pd.merge(ocr_log,z1,on=["InvoiceAmount","DocumentNumber"],how="left",indicator=True)

In [2356]:
ocr_bsak_match["_merge"].value_counts()

left_only     237664
both             935
right_only         0
Name: _merge, dtype: int64

In [2359]:
#ocr_bsak_match.Check.value_counts()

In [2360]:
ocr_bsak_match.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,FileName,DocumentNumber,InvoiceAmount,Check,_merge
0,1,1,0,0,0,0,0,0,2550,3300,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaN,left_only
1,2,1,1,0,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaN,left_only
2,3,1,1,1,0,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaN,left_only
3,4,1,1,1,1,0,237,3081,507,53,-1,NaN,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaN,left_only
4,5,1,1,1,1,1,237,3087,238,44,19,hy,/home/sambid/results/5100011564.pdf1.png,5.100012e+09,,NaN,left_only


In [2362]:
#no_match.shape

In [2361]:
ocr_bsak_match.to_csv("/datarepo/sambid/ocr_log_set_abinbev_tabular_data/ocr_inv_amt.csv",index=False)